In [ ]:
from src.config.config_loader import ConfigLoader
from src.agents.sql_agent import SQLAgent
from src.agents.metadata_enhancer_agent import MetadataAgent
from src.factories.database import DatabaseFactory
from src.factories.llm import LLMFactory

# 1. Carica la configurazione
config = ConfigLoader.load_config('config.yaml')

# 2. Crea le dipendenze di base
db = DatabaseFactory.create_connector(config.database)
db.connect()
llm = LLMFactory.create_handler(config.llm)
metadata_retriever = DatabaseFactory.create_metadata_retriever(config.database, db)

# 3. Crea gli agenti
sql_agent = SQLAgent(
    db=db,
    llm_manager=llm,
    metadata_retriever=metadata_retriever,
    schema_name=config.database.schema,
    prompt_config=config.prompt,
    vector_store=None,  # opzionale per uso notebook
    language=config.llm.language
)

metadata_agent = MetadataAgent(llm_handler=llm)

# 4. Usa SQLAgent per query
question = "Mostrami i 5 giochi più venduti"
response = sql_agent.run(question)

if response.success:
    print("Query:", response.query)
    print("\nSpiegazione:", response.explanation)
    print("\nRisultati:", response.results[:2], "...")  # primi 2 per brevità
else:
    print("Errore:", response.error)

# 5. Usa MetadataAgent per analizzare lo schema
table_metadata = metadata_retriever.get_all_tables_metadata()
metadata_response = metadata_agent.run(table_metadata)

if metadata_response.success:
    # Mostra l'enhancement di una tabella
    table_name = list(metadata_response.enhanced_metadata.keys())[0]
    enhanced = metadata_response.enhanced_metadata[table_name]

    print(f"\nAnalisi della tabella {table_name}:")
    print("Descrizione:", enhanced.description)
    print("Keywords:", enhanced.keywords)
    print("Importance Score:", enhanced.importance_score)
else:
    print("Errore nell'enhancement:", metadata_response.error)

# 6. Chiudi le connessioni
db.close()